In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# from bs4 import BeautifulSoup
# import re
# import pandas as pd
import os
import time
import re
import shutil

In [ ]:
url = 'https://www.symmap.org/browse/'
dropdown_target = 'Disease'

# Open Firefox and Nav to URL
## NOTE -- Need to modify certain prefs for your system
Will be noted with comments

## Geckodriver
https://github.com/mozilla/geckodriver/releases  
I downloaded and unzipped one of the pre-compiled geckodrivers for my Windows system

In [ ]:
fp = webdriver.FirefoxProfile()
fp.set_preference('browser.download.folderList', 2) 
fp.set_preference('browser.download.manager.showWhenStarting', False)
# Set the default download dir
fp.set_preference('browser.download.dir', r"C:\\selenium\\")
fp.set_preference('browser.helperApps.neverAsk.openFile', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
fp.set_preference('browser.helperApps.neverAsk.saveToDisk', 'text/csv,application/x-msexcel,application/excel,application/x-excel,application/vnd.ms-excel,image/png,image/jpeg,text/html,text/plain,application/msword,application/xml')
fp.set_preference('browser.helperApps.alwaysAsk.force', False)
fp.set_preference('browser.download.manager.alertOnEXEOpen', False)
fp.set_preference('browser.download.manager.focusWhenStarting', False)
fp.set_preference('browser.download.manager.useWindow', False)
fp.set_preference('browser.download.manager.showAlertOnComplete', False)
fp.set_preference('browser.download.manager.closeWhenDone', False)
# Point to the geckodriver you downloaded on your local
driver = webdriver.Firefox(fp, executable_path=r'C:\Users\alexl\Downloads\geckodriver-v0.26.0-win64\geckodriver.exe')
driver.window_handles
driver.switch_to_window(driver.window_handles[0])
driver.get(url)

# Find the Dropdown and Click to Open
# Select 'Disease' & Click

In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

In [ ]:
# Click the dropdown
driver.find_element_by_xpath(r"//button[@title='Herb']").click()

# Wait until the list loads, then click on the fifth list item which is 'diseases'
target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul/li[@data-original-index='5']")))
target.click()

# Loop Through Pagination & Get All Disease ID Links
# Ignore this because I couldn't solve pagination quickly
Skip down to the workaround

In [ ]:
# # Show max (50) results instead of default 15
# # Click the dropdown
# driver.find_element_by_xpath(r"//button[@class='btn btn-default dropdown-toggle']").click()

# # Wait until the list loads, then click on the third list item which is 50
# target = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, "//ul/li[@role='menuitem'][3]")))
# target.click()

In [ ]:
# Pagination Test
# TODO


In [ ]:
# # Build the name and link dict
# disease_id_dict = dict()

# links = driver.find_elements_by_xpath("//td//a[@href]")
# for link in links:
#     # Store links to disease pages as {disease_id: href} pairs
#     disease_id_name = re.findall(pattern=r"([^\/]+$)", string=link.get_attribute("href"))[0]
#     disease_id_dict[disease_id_name] = link.get_attribute("href")

In [ ]:
# disease_id_dict

# Workaround
## Screw the Pagination and Pull What Exists as of Now
Disease IDs go from 00001 - 05235. We'll just programatically generate the links.

In [ ]:
disease_id_dict = dict()

url_head = "https://www.symmap.org/detail/SMDE0"
max_id = 5235

for i in range(1, max_id + 1):
    # Store links to disease pages as {disease_id: href} pairs
    id_len = len(str(i))
    padding_size = 4 - id_len
    disease_id_name = 'SMDE0' + padding_size * str(0) + str(i)
    disease_id_dict[disease_id_name] = url_head + padding_size * str(0) + str(i)

In [ ]:
disease_id_dict

# Click the Disease Link and Download

In [ ]:
# File save & rename paths
# Selenium can't handle file renaming the way this is set up,
# so the workaround is to rename and move the file to another
# directory
filepath = r'c:/selenium'
destination = r'c:/selenium/named_files/'

# Time Outs to Prevent Banhammer
Each run adds a total of 5 seconds of sleeptime to avoid hammering the server and getting banned. With 5,335 files to pull, this requires over 7 hours to get the docs. May want to play around or split the pull across multiple ppl by assigning tranches of IDs.

In [ ]:
# Loop through the dict
for name, link in disease_id_dict.items():
    # Add some wait time to avoid hammering server
    time.sleep(1)
    
    # Pull the link
    driver.get(link)
    
    # Zzzzzzz
    time.sleep(2)
    
    # Find and click the download button
    driver.find_element_by_xpath(r"//button[@id='dl-btn']").click()
    
    # Look for the newest file and store the name
    filename = max([filepath +"/"+ f for f in os.listdir(filepath)], key=os.path.getctime)
    
    # Use shutil.move() to rename and move that file to the final dir
    shutil.move(os.path.join(filepath, filename), os.path.join(destination, name + '.csv'))
    
    # Zzzzzzzz
    time.sleep(2)